In [11]:
"""Simulation script."""

import os
import sys
sys.path.append("../") # go to parent dir
import time
import pathlib
import logging
import numpy as np
from mpi4py import MPI
from scipy.sparse import linalg as spla
from dedalus.tools.config import config
from simple_sphere import SimpleSphere
import equations

# Logging and config
logger = logging.getLogger(__name__)
STORE_LU = config['linear algebra'].getboolean('store_LU')
PERMC_SPEC = config['linear algebra']['permc_spec']
USE_UMFPACK = config['linear algebra'].getboolean('use_umfpack')

# Discretization parameters
L_max = 127  # Spherical harmonic order
S_max = 4  # Spin order (leave fixed)

# Model parameters
Lmid = 4   #gives 1/10 as characteristic diameter for the vortices
kappa = 1    #spectral injection bandwidth
gamma = 1  # surface mass density
fspin = 0

### calculates e0, e1, e2 from Lmid and kappa
a = 0.25*(Lmid**2*kappa**2 - 0.5*(2*np.pi*Lmid+1)**2)**2 + 17*17/16 - (34/16)*(2*np.pi*Lmid+1)**2
b = (17/4 - 0.25*(2*np.pi*Lmid+1)**2)**2
c = 1/(17/4 - 0.25*(2*np.pi*Lmid + 1)**2 - 2)
e0 = a*c/(a-b)
e1 = 2*np.sqrt(b)*c/(a-b)
e2 = c/(a-b)

params = [gamma, e0, e1, e2, fspin]

# Integration parameters
Amp = 1e-2  # initial noise amplitude
factor = 0.5   #controls the time step below to be 0.5/(100*Lmid^2), which is 0.5/100 of characteristic vortex dynamics time
dt = factor/(100)
n_iterations = int(100/factor)# total iterations. Change 10000 to higher number for longer run!
n_output = int(10/factor)  # data output cadence
n_clean = 10
output_folder = 'output_garbage'  # data output folder

# Find MPI rank
comm = MPI.COMM_WORLD
rank = comm.rank

# Domain
start_init_time = time.time()
simplesphere = SimpleSphere(L_max, S_max)
domain = simplesphere.domain

# Model
model = equations.ActiveMatterModel(simplesphere, params)
state_system = model.state_system

# Matrices
# Combine matrices and perform LU decompositions for constant timestep
A = []
for dm, m in enumerate(simplesphere.local_m):
    # Backward Euler for LHS
    Am = model.M[dm] + dt*model.L[dm]
    if STORE_LU:
        Am = spla.splu(Am.tocsc(), permc_spec=PERMC_SPEC)
    A.append(Am)



2019-05-31 16:37:27,752 equations 0/1 INFO :: Building matrix 0
2019-05-31 16:37:27,784 equations 0/1 INFO :: Building matrix 1
2019-05-31 16:37:27,817 equations 0/1 INFO :: Building matrix 2
2019-05-31 16:37:27,848 equations 0/1 INFO :: Building matrix 3
2019-05-31 16:37:27,873 equations 0/1 INFO :: Building matrix 4
2019-05-31 16:37:27,904 equations 0/1 INFO :: Building matrix 5
2019-05-31 16:37:27,932 equations 0/1 INFO :: Building matrix 6
2019-05-31 16:37:27,959 equations 0/1 INFO :: Building matrix 7
2019-05-31 16:37:27,989 equations 0/1 INFO :: Building matrix 8
2019-05-31 16:37:28,020 equations 0/1 INFO :: Building matrix 9
2019-05-31 16:37:28,045 equations 0/1 INFO :: Building matrix 10
2019-05-31 16:37:28,068 equations 0/1 INFO :: Building matrix 11
2019-05-31 16:37:28,098 equations 0/1 INFO :: Building matrix 12
2019-05-31 16:37:28,131 equations 0/1 INFO :: Building matrix 13
2019-05-31 16:37:28,157 equations 0/1 INFO :: Building matrix 14
2019-05-31 16:37:28,188 equations 0

2019-05-31 16:37:31,263 equations 0/1 INFO :: Building matrix 126
2019-05-31 16:37:31,287 equations 0/1 INFO :: Building matrix 127


In [23]:
phi_flat = simplesphere.phi_grid.ravel()
theta_flat = simplesphere.global_theta_grid.ravel()
theta, phi = np.meshgrid(theta_flat, phi_flat)
v = model.v
sh = v.component_fields[1]['g'].shape
v.component_fields[1]['g'] = np.sin(theta)


In [21]:
# Initial conditions
# Add random perturbations to the velocity coefficients
v = model.v
rand = np.random.RandomState(seed=42+rank)
for dm, m in enumerate(simplesphere.local_m):
    shape = v.coeffs[dm].shape
    noise = rand.standard_normal(shape)
    phase = rand.uniform(0,2*np.pi,shape)
    v.coeffs[dm] = Amp * noise*np.exp(1j*phase)
state_system.pack_coeffs()

# Setup outputs
file_num = 1
if not os.path.exists(output_folder):
    os.mkdir(output_folder)


In [18]:
sh

(256, 128)

In [22]:
phi.shape

(256, 128)